In [10]:
from ipywidgets import interact, widgets
from IPython.display import display, clear_output
from boto3.dynamodb.conditions import Key, Attr
from decimal import Decimal
from functools import reduce
from pymongo import MongoClient
import pandas as pd
import logging
import re
import boto3


In [11]:
client = MongoClient('localhost', 27017)
db = client['351Project']
collection = db['profiles']

# establishes dynamo connection
dynamodb = boto3.resource('dynamodb', region_name='us-west-1') # region name must match aws 
table = dynamodb.Table('video_game_profiles') 

In [12]:
def view_mongo():
    
    # to have labels in HTMl so it fits on the screen
    def labeled_input(description, widget):
        return widgets.VBox([widgets.Label(value=f"{description}"), widget])
        
    # widgets that will be displayed
    view_choice = widgets.Dropdown(
        options = ['All Profiles', 'Search Profile by Username'],
    )
    labeled_view_choice = labeled_input('View by: ', view_choice)

    username = widgets.Text(
        value = '',
        placeholder = 'Enter username',
        disabled = False,

    )
    labeled_username = labeled_input('Username: ', username)
    labeled_username.layout.visibility = 'hidden'

        
    options = [ 'all', 'username', 'email', 'games_played', 'wins', 'losses',
                  'kills', 'number_of_friends', 'favorite_weapon', 'money_spent', 
                  'favorite_character']
    attributes = widgets.SelectMultiple(
        options = options,
        value = ['all'],
        disabled = False
    )
    labeled_attributes = labeled_input("Attributes(hold command to select multiple): ", attributes)

    search_button = widgets.Button(
        description = 'View',
        button_style = '',
        tooltip = 'Click to search for a profile',
        icon = 'view'
    )
    
    clear_button = widgets.Button(
        description = 'Clear', 
        button_style = '', 
        tooltip = 'Click to clear search', 
        icon = 'remove')
 
    
    output = widgets.Output()

    # function so user can't select all along with the other options, only all, or only the other options
    def select_all(change):
        if 'all' in change['new']:
            if len(change['new']) > 1:
                if 'all' in change['old']:
                    attributes.value = tuple(set(change['new']) - {'all'})
                else:
                    attributes.value = tuple(options[1:])
            else:
                attributes.value = ('all',)
                
    attributes.observe(select_all, 'value')

    # function that allows user to view all profiles or view profile by username
    # also allows user to select which attributes they would like to view or if they would like to view all of them
    def valid_search(submission):
        with output:
            output.clear_output()
            selected_attributes = attributes.value
            if 'all' in selected_attributes:
                selected_attributes = options[1:]

            fields = {field: True for field in selected_attributes} 
            fields['_id'] = 0
            if view_choice.value == 'All Profiles':
                profiles = collection.find({}, fields)
                for profile in profiles:
                    display = ', '.join([f"{key}: {value}" for key, value in profile.items() if key != '_id'])
                    print(display)
            elif view_choice.value == "Search Profile by Username" and username.value:
                profile = collection.find_one({"username": username.value}, fields)
                if profile:
                    out = ', '.join([f"{key}: {value}" for key, value in profile.items() if key != '_id'])
                    print(out)
                else:
                    print("Username does not exist, Please Enter another one")
            else:
                print("Invalid Choice or No username Entered")

    search_button.on_click(valid_search)
    
    # clear function to reset the searches when click is cleared
    def clear(click):
        with output:
            clear_output()


    clear_button.on_click(clear)

    # function that only shows the username search widget when 'Search Profile by Username is selected'
    def username_visibility(change):
        if change['new'] == 'Search Profile by Username':
            labeled_username.layout.visibility = 'visible'
            username.layout.visibility = 'visible'
        else:
            labeled_username.layout.visibility = 'hidden'
            username.layout.visibility = 'hidden'
            username.value = ''
    view_choice.observe(username_visibility, names = 'value')

    # to display the GUI
    buttons = widgets.HBox([search_button, clear_button])
    layout = widgets.VBox([labeled_view_choice, labeled_username, labeled_attributes, buttons, output])
    display(layout)


In [13]:
def view_dynamo():

    # Function to create labeled input widgets
    def labeled_input(description, widget):
        return widgets.VBox([widgets.Label(value=f"{description}"), widget])

    # Widgets for user interface
    view_choice = widgets.Dropdown(
        options=['All Profiles', 'Search Profile by Username'],
    )
    labeled_view_choice = labeled_input('View by: ', view_choice)

    username = widgets.Text(
        value='',
        placeholder='Enter username',
        disabled=False,
    )
    labeled_username = labeled_input('Username: ', username)
    labeled_username.layout.visibility = 'hidden'

    options = ['all', 'username', 'email', 'games_played', 'wins', 'losses',
               'kills', 'number_of_friends', 'favorite_weapon', 'money_spent',
               'favorite_character']
    attributes = widgets.SelectMultiple(
        options=options,
        value=['all'],
        disabled=False
    )
    labeled_attributes = labeled_input("Attributes(hold command to select multiple): ", attributes)

    search_button = widgets.Button(
        description='View',
        button_style='',
        tooltip='Click to search for a profile',
        icon='view'
    )

    clear_button = widgets.Button(
        description='Clear',
        button_style='',
        tooltip='Click to clear search',
        icon='remove')

    output = widgets.Output()

    # Function to handle attribute selection
    def select_all(change):
        if 'all' in change['new']:
            if len(change['new']) > 1:
                if 'all' in change['old']:
                    attributes.value = tuple(set(change['new']) - {'all'})
                else:
                    attributes.value = tuple(options[1:])
            else:
                attributes.value = ('all',)

    attributes.observe(select_all, 'value')

    # function to perform valid search. if "all" is chosen, table scan is used and all results are returned. if "search profile by
    # username" is chosen, table query is used instead
    def valid_search(submission):
        with output:
            output.clear_output()
            selected_attributes = attributes.value
            if 'all' in selected_attributes:
                selected_attributes = options[1:]

            fields = selected_attributes
            if view_choice.value == 'All Profiles':
                response = table.scan(ProjectionExpression=', '.join(fields))
                for item in response['Items']:
                    display = ', '.join([f"{key}: {value}" for key, value in item.items()])
                    print(display)
            elif view_choice.value == "Search Profile by Username" and username.value:
                response = table.query(
                    KeyConditionExpression=Key('username').eq(username.value),
                    ProjectionExpression=', '.join(fields)
                )
                for item in response['Items']:
                    display = ', '.join([f"{key}: {value}" for key, value in item.items()])
                    print(display)
                if not response['Items']:
                    print("Username does not exist, Please Enter another one")
            else:
                print("Invalid Choice or No username Entered")

    search_button.on_click(valid_search)

    # Function to clear the output
    def clear(click):
        with output:
            clear_output()

    clear_button.on_click(clear)

    # Function to handle visibility of username input
    def username_visibility(change):
        if change['new'] == 'Search Profile by Username':
            labeled_username.layout.visibility = 'visible'
            username.layout.visibility = 'visible'
        else:
            labeled_username.layout.visibility = 'hidden'
            username.layout.visibility = 'hidden'
            username.value = ''

    view_choice.observe(username_visibility, names='value')

    # Display the GUI
    buttons = widgets.HBox([search_button, clear_button])
    layout = widgets.VBox([labeled_view_choice, labeled_username, labeled_attributes, buttons, output])
    display(layout)


In [14]:
def change_mongo():
    
    # to have labels in HTMl so it fits on the screen
    def labeled_input(description, widget):
        return widgets.VBox([widgets.Label(value=f"{description}"), widget])

    # widgets that will appear on the main menu to give user the option to select "Insert", "Delete",
    # or "Modify"
    action = widgets.Dropdown(options = ["Insert", "Delete", "Modify"])
    labeled_action = labeled_input("Action: ", action)
    
    submit = widgets.Button(description = "Submit")
    
    clear_button = widgets.Button(
        description = 'Clear', 
        button_style = '', 
        tooltip = 'Click to clear search', 
        icon = 'remove')
    
    output = widgets.Output()

    # function so that based on which option is selected, the correct action will occur
    def choice(click):
        with output:
            clear_output()
            if action.value == "Insert":
                valid_insert()
            elif action.value == "Delete":
                valid_delete()
            elif action.value == "Modify":
                valid_modify()

    # valid_insert is function for if the user selects insert
    def valid_insert():

        # widgets that will be used if user selects "Insert", have a form widget and message widget
        form_output = widgets.Output()
        message_output = widgets.Output()
        
        username = widgets.Text()
        email = widgets.Text()
        wins = widgets.IntText(min = 0)
        losses = widgets.IntText(min = 0)
        kills = widgets.IntText(min = 0)
        number_of_friends = widgets.IntText(min = 0)
        favorite_weapon = widgets.Text()
        money_spent = widgets.FloatText(min = 0.0)
        favorite_character = widgets.Text()

        # function so that user can insert a new profile, user is able to enter all 10 fields to make the
        # new profile; function ensures that username and email cannot be left blank or a duplicate entry and that
        # numerical values have to be greater than or equal to 0, wins, losses, number of friends and kills must be ints,
        # money spent can be a float
        def insert(submission):
            with message_output:
                message_output.clear_output()
                if not username.value.strip() or not email.value.strip():
                    print("Username and email cannot be left blank.")
                    return
                #profile_inserted = True
                try: 
                    if any(int(value) < 0 for value in [wins.value, losses.value, kills.value, number_of_friends.value]):
                        print("Wins, losses, kills and number of friends must be an integer greater than or equal to 0.")
                        return
                        
                    if float(money_spent.value) < 0 :
                        print("Money spent must be a number greater than or equal to 0.")
                        return
                except ValueError:
                    print("Non numeric value entered for wins, losses, kills, number of friends, or money spent")
                    
                if collection.find_one({"username": username.value}) or collection.find_one({"email": email.value}): 
                    print("Username or Email already in use. Please enter a new one.")
                    return
                profile_data = {
                    "username": username.value,
                    "email": email.value,
                    "wins": wins.value,
                    "losses": losses.value,
                    "games_played": wins.value+losses.value,
                    "kills": kills.value,
                    "number_of_friends": number_of_friends.value,
                    "favorite_weapon": favorite_weapon.value,
                    "money_spent": money_spent.value,
                    "favorite_character": favorite_character.value,
                  
                }
                collection.insert_one(profile_data)
                print("Profile Inserted")
                
        insert_button = widgets.Button(description = "Insert")
        insert_button.on_click(insert)

        
        # widgets that will be displayed when insert is selected
        insert_items = [labeled_input('Username: ', username), 
                  labeled_input('Email: ', email), 
                  labeled_input('Wins: ', wins),
                  labeled_input('Losses: ', losses), 
                  labeled_input('Kills: ', kills), 
                  labeled_input('Number of Friends: ', number_of_friends), 
                  labeled_input('Favorite Weapon: ', favorite_weapon), 
                  labeled_input('Money Spent: ', money_spent),
                  labeled_input('Favorite Character: ', favorite_character),
                  insert_button]
        
        display(form_output)
        for item in insert_items:
            display(item)
        display(message_output)

    # valid_delete is function for if the user selects delete
    def valid_delete():

        # widgets that will be used if user selects delete
        output = widgets.Output()   
        output.clear_output(wait = True)
        numeric_conditions_container = widgets.VBox([])
        favorite_conditions_container = widgets.VBox([])
        
        delete_button = widgets.Button(description =  "Delete")

        username = widgets.Text()
        labeled_username = labeled_input("Username: ", username)

        options = ['all', 'wins', 'losses', 'games_played', 'kills', 'number_of_friends',
                       'money_spent', 'favorite_character']
        fields_to_delete = widgets.SelectMultiple(
            options = options,
            value = ['all'],
            disabled = False
        )
        labeled_fields_to_delete = labeled_input('Field(s) to Delete: ', fields_to_delete)
        
        
        number_of_numeric_conditions = widgets.Dropdown(
            options = [0,1,2,3,4,5,6],
            value = 0,
            disabled = False,
        )
        labeled_number_of_numeric_conditions = labeled_input("Number of Numeric Conditions: ", number_of_numeric_conditions)

        
        number_of_favorite_conditions = widgets.Dropdown(
            options = [0,1,2],
            value = 0,
            disabled = False,
        )
        labeled_number_of_favorite_conditions = labeled_input("Number of Favorite Conditions: ", number_of_favorite_conditions)

        operator = widgets.Dropdown(
            options = ['Select', 'AND', 'OR'],
            value = "Select",
            disabled = False
        )
        labeled_operator = labeled_input("Operator: ", operator)
       
        # this function allows user to select how many numeric conditions they would like; numeric fields include
        # 'wins', 'losses', 'games played', 'kills', 'number of friends', and 'money spent'; function will 
        # display number of numeric condition widgets according to the number selected by user. 
        def num_of_numeric_conditions(change):
            children = []
            for i in range(change['new']):
                field_choice = widgets.Dropdown(
                    options = ['select', 'wins', 'losses', 'games_played', 'kills', 
                               'number_of_friends', 'money_spent'],
                    disabled = False
                )
                labeled_field_choice = labeled_input("Field Choice: ", field_choice)
                
                operator_choice = widgets.Dropdown(
                    options = ['select', '=', '>', '>=', '<', '<='],
                    disabled = False
                )
                labeled_operator_choice = labeled_input("Operator Choice: ", operator_choice)
                
                value = widgets.FloatText(
                value = 0,
                disabled = False
                )
                labeled_value = labeled_input("Value: ", value)
                
                condition_row = widgets.HBox([labeled_field_choice, labeled_operator_choice, labeled_value])
                children.append(condition_row)
            numeric_conditions_container.children = children

        number_of_numeric_conditions.observe(num_of_numeric_conditions, names = 'value')
    
 
        # this function allows user to select how many favorite conditions they would like; favorite fields include
        # favorite character and favorite weapon; function will 
        # display number of favorite condition widgets according to the number selected by user
        def num_of_favorite_conditions(change):
            children = []
            for i in range(change['new']):
                field = widgets.Dropdown(
                    options = ['select', 'favorite_character', 'favorite_weapon'],
                    disabled = False
                )
                labeled_field = labeled_input("Field Choice: ", field)
                
                operation = widgets.Dropdown(
                    options = ['select', '=', '≠'],
                    disabled = False
                )
                labeled_operation = labeled_input("Operation Choice: ", operation)
                
                favorite = widgets.Text(
                    disabled = False
                )
                labeled_favorite = labeled_input("Value: ", favorite)
                
                condition_row = widgets.HBox([labeled_field, labeled_operation, labeled_favorite])
                children.append(condition_row)
            favorite_conditions_container.children = children

        number_of_favorite_conditions.observe(num_of_favorite_conditions, names = 'value')

 
        # delete function will delete the profile(s) or column(s) in database; deletion is based on either
        # the username entered by the user or based on the numeric and/or the favorite conditions; numeric
        # conditions give user the option to select the numeric fields described above, as well as one of the conditions,
        # <, >, <=, >=, =, and the value to compare and delete based on the conditions; favorite conditions
        # give the user the option to select the favorite fields as described above, as well as one of the conditions,
        # = or ≠ and the value to compare to and delete based on the conditions; user can select multiple conditions
        # from either favorite or numeric and select AND or OR to join them; function also gives user the option to
        # select how many fields to delete, or all to delete an entire profile; finally, function will tell user 
        # how many profile(s) or column(s) were deleted
        def delete(submission):
            with output:
                clear_output()
                query_conditions = []

                if username.value:
                    query_conditions.append({"username": username.value})

                for search_condition in numeric_conditions_container.children + favorite_conditions_container.children:
                    field, condition, value = [c.children[1].value for c in search_condition.children]
                    if field != 'select' and condition != 'select':
                        query_conditions.append(numeric_query(field, condition, value))

                logical_operator = {"AND": "$and", "OR": "$or"}.get(operator.value, "$and")

                query = {logical_operator:query_conditions} if query_conditions else{}
                
                
                if 'all' in fields_to_delete.value:
                    result = collection.delete_one({logical_operator: query_conditions})
                    print(f"Deleted {result.deleted_count} profiles")
                else:
                    if not query_conditions:
                        print("Specify username or condition to delete.")
                        return
                    fields_to_unset = {field: "" for field in fields_to_delete.value if field != 'all'}

                    if fields_to_unset:
                        result = collection.update_many(query, {"$unset": fields_to_unset})
                    if result.modified_count > 0:
                        print(f"Field(s) {', '.join(fields_to_unset)} deleted from {result.modified_count} profile(s).")
                    else:
                        print("No fields were deleted")
            
        # function so that user can not select 'all' with other options for field(s) to delete to avoid
        # redundancy; user can either select other fields together or all by itself.
        def select_all(change):
            if 'all' in change['new']:
                if len(change['new']) > 1:
                    if 'all' in change['old']:
                        fields_to_delete.value = tuple(set(change['new']) - {'all'})
                    else:
                        fields_to_delete.value = tuple(options[1:])
                else:
                    fields_to_delete.value = ('all',)
            
        fields_to_delete.observe(select_all, 'value')

 
        # widgets that will be displayed when delete is selected
        delete_button.on_click(delete)
        show_widgets = [
            labeled_fields_to_delete,
            labeled_username,
            labeled_number_of_numeric_conditions,
            numeric_conditions_container,
            labeled_number_of_favorite_conditions,
            favorite_conditions_container,
            labeled_operator,
            delete_button,
            output
        ]
        display(*show_widgets)

    
    # valid_modify is function for if the user selects modify
    def valid_modify():

        # widgets that will be used if user selects modify
        output = widgets.Output()
        numeric_conditions_container = widgets.VBox([])
        favorite_conditions_container = widgets.VBox([])
        modify_button = widgets.Button(description = "Modify")
        
        username = widgets.Text()
        labeled_username = labeled_input("Username: ", username)

        options = ['username', 'email', 'wins', 'losses', 'kills', 'number_of_friends',
                       'money_spent', 'favorite_character', 'favorite_weapon']
        fields_to_modify = widgets.SelectMultiple(
            options = options,
            disabled = False
        )
        labeled_fields_to_modify = labeled_input('Column(s) to Modify: ', fields_to_modify)

        number_of_numeric_conditions = widgets.Dropdown(
            options = [0,1,2,3,4,5,6],
            value = 0,
            disabled = False,
        )
        labeled_number_of_numeric_conditions = labeled_input("Number of Numeric Conditions: ", number_of_numeric_conditions)

        
        number_of_favorite_conditions = widgets.Dropdown(
            options = [0,1,2],
            value = 0,
            disabled = False,
        )
        labeled_number_of_favorite_conditions = labeled_input("Number of Favorite Conditions: ", number_of_favorite_conditions)

        operator = widgets.Dropdown(
            options = ['Select', 'AND', 'OR'],
            value = "Select",
            disabled = False
        )
        labeled_operator = labeled_input("Operator: ", operator)

        modify_fields_container = widgets.VBox([])

        # function so that user can select which field(s) they want to change and a textbox will
        # appear where they can input the new value for the corresponding field(s) they want to change
        def fields_modify(change):
            children = []
            for field in change['new']:
                value_input = widgets.Text()
                labeled_value_input = labeled_input(f"New {field}: ", value_input)
                children.append(labeled_value_input)
            modify_fields_container.children = children

        fields_to_modify.observe(fields_modify, names = 'value')

        # this function(like the one for delete) allows user to select how many numeric conditions they would like; numeric fields include
        # 'wins', 'losses', 'games played', 'kills', 'number of friends', and 'money spent'; function will 
        # display number of numeric condition widgets according to the number selected by user. 
        def num_of_numeric_conditions(change):
            children = []
            for i in range(change['new']):
                field_choice = widgets.Dropdown(
                    options = ['select', 'wins', 'games_played', 'losses', 'kills', 
                               'number_of_friends', 'money_spent'],
                    disabled = False
                )
                labeled_field_choice = labeled_input("Field Choice: ", field_choice)
                
                operator_choice = widgets.Dropdown(
                    options = ['select', '=', '>', '>=', '<', '<='],
                    disabled = False
                )
                labeled_operator_choice = labeled_input("Operator Choice: ", operator_choice)
                
                value = widgets.FloatText(
                value = 0,
                disabled = False
                )
                labeled_value = labeled_input("Value: ", value)
                
                condition_row = widgets.HBox([labeled_field_choice, labeled_operator_choice, labeled_value])
                children.append(condition_row)
            numeric_conditions_container.children = children

        number_of_numeric_conditions.observe(num_of_numeric_conditions, names = 'value')

        # this function(like the one for delete) allows user to select how many favorite conditions they would like; favorite fields include
        # favorite character and favorite weapon; function will display number of favorite condition 
        # widgets according to the number selected by user
        def num_of_favorite_conditions(change):
            children = []
            for i in range(change['new']):
                field = widgets.Dropdown(
                    options = ['select', 'favorite_character', 'favorite_weapon'],
                    disabled = False
                )
                labeled_field = labeled_input("Field Choice: ", field)
                
                operation = widgets.Dropdown(
                    options = ['select', '=', '≠'],
                    disabled = False
                )
                labeled_operation = labeled_input("Operation Choice: ", operation)
                
                favorite = widgets.Text(
                    disabled = False
                )
                labeled_favorite = labeled_input("Value: ", favorite)
                
                condition_row = widgets.HBox([labeled_field, labeled_operation, labeled_favorite])
                children.append(condition_row)
            favorite_conditions_container.children = children

        number_of_favorite_conditions.observe(num_of_favorite_conditions, names = 'value')


        # modify function will modify the profile(s) or column(s) in database; modification is based on either
        # the username entered by the user or based on the numeric and/or the favorite conditions; numeric
        # conditions give user the option to select the numeric fields described above, as well as one of the conditions,
        # <, >, <=, >=, =, and the value to compare and modify based on the conditions; favorite conditions
        # give the user the option to select the favorite fields as described above, as well as one of the conditions,
        # = or ≠ and the value to compare to and modify based on the conditions; user can select multiple conditions
        # from either favorite or numeric and select AND or OR to join them; user will also be given option
        # to select which fields they want to modify and input the new values.
        # finally, function will tell user how many profile(s)/column(s) were modified
        def modify(submission):
            with output:
                clear_output()
                query_conditions = []

                if username.value:
                    query_conditions.append({"username": username.value})

                for search_condition in numeric_conditions_container.children + favorite_conditions_container.children:
                    field, condition, value = [c.children[1].value for c in search_condition.children]
                    if field != 'select' and condition != 'select':
                        query_conditions.append(numeric_query(field, condition, value))

                logical_operator = {"AND": "$and", "OR": "$or"}.get(operator.value, "$and")

                query = {logical_operator:query_conditions} if query_conditions else{}
            
                if not query_conditions:
                    print("Specify username or condition to delete.")
                    return

                modify_values = {}

                for i in modify_fields_container.children:
                    label_widget = i.children[0]
                    text_widget = i.children[1]
                    
                    field = label_widget.value.replace("New", "").replace(":", "").strip().lower()
                    original_value = text_widget.value.strip()
                    modify_values[field] = original_value if original_value else None

                    if field in ['username', 'email'] and not original_value:
                        print(f"Cannot leave '{field}' blank")
                        return
                        
                    if field in ['wins', 'losses',  'kills', 'number_of_friends']:
                        try:
                            value = int(original_value)
                            if value < 0:
                                print(f"New Value for {field} cannot be negative.")
                                return

                        except ValueError:
                            print(f"New Value for {field} must be a integer greater than or equal to 0.")
                            return
                    elif field in ['money_spent']:
                        try:
                            value = int(original_value)
                            if value < 0:
                                print("New Value for 'money_spent' cannot be negative.")
                                return

                        except ValueError:
                            print(print("New Value for 'money_spent' must be a number greater than or equal to 0."))
                            return
                    else:
                        value = original_value
                            
                    modify_values[field.strip()] = value

                if 'wins' in modify_values or 'losses' in modify_values:
                    current_profile = collection.find_one({"username": username.value})
                    if current_profile:
                        current_wins = modify_values.get('wins', current_profile.get('wins', 0))
                        current_losses = modify_values.get('losses', current_profile.get('losses', 0))
                        modify_values['games_played'] = current_wins+current_losses
                        

                
                if modify_values:
                    result = collection.update_many(query, {"$set": modify_values})
                    if result.modified_count > 0:
                        print(f"Modified {result.modified_count} profile(s)")
                    else:
                        print("No profiles were modified")
                         
        modify_button.on_click(modify)

        # widgets that will be displayed when modify is selected
        show_widgets = [
            labeled_fields_to_modify,
            labeled_username,
            labeled_number_of_numeric_conditions,
            numeric_conditions_container,
            labeled_number_of_favorite_conditions,
            favorite_conditions_container,
            labeled_operator,
            modify_fields_container,
            modify_button,
            output
        ]
        display(*show_widgets)
    
    # used to translate the symbols to the Mongo syntax
    def numeric_query(field, condition, value):
        operators = {
            ">":"$gt",
            "<":"$lt",
            ">=":"$gte",
            "<=":"$lte",
            "=":"$eq",
            "≠":"$ne"
        }
        return {field: {operators[condition]: value}}
        
        num_of_condition({'new': 1})

    # delete and modify button
    #delete_button = widgets.Button(description = "Delete")
    #modify_button = widgets.Button(description = "Modify")
    
   
    # to clear when clicked on 
    def clear(click):
        with output:
            clear_output()
       
    submit.on_click(choice)
    clear_button.on_click(clear)

    # display for the main menu
    buttons = widgets.HBox([submit, clear_button])
    display(labeled_action, buttons, output)

In [15]:
def change_dynamo():

    def labeled_input(description, widget):
        return widgets.VBox([widgets.Label(value=f"{description}"), widget])

    action = widgets.Dropdown(options=["Insert", "Delete", "Modify"])
    labeled_action = labeled_input("Action: ", action)

    submit = widgets.Button(description="Submit")
    clear_button = widgets.Button(description='Clear', button_style='', tooltip='Click to clear search', icon='remove')
    output = widgets.Output()

    def choice(click):
        with output:
            clear_output()
            if action.value == "Insert":
                valid_insert()
            elif action.value == "Delete":
                valid_delete()
            elif action.value == "Modify":
                valid_modify()

    def valid_insert():
        form_output = widgets.Output()
        message_output = widgets.Output()

        username = widgets.Text()
        email = widgets.Text()
        wins = widgets.IntText(min=0)
        losses = widgets.IntText(min=0)
        kills = widgets.IntText(min=0)
        number_of_friends = widgets.IntText(min=0)
        favorite_weapon = widgets.Text()
        money_spent = widgets.FloatText(min=0.0)
        favorite_character = widgets.Text()

        def insert(submission):
            with message_output:
                message_output.clear_output()
                if not username.value.strip() or not email.value.strip():
                    print("Username and email cannot be left blank.")
                    return
    
                try:
                    if any(int(value) < 0 for value in [wins.value, losses.value, kills.value, number_of_friends.value]):
                        print("Wins, losses, kills, and number of friends must be integers greater than or equal to 0.")
                        return
    
                    if money_spent.value < 0:
                        print("Money spent must be a number greater than or equal to 0.")
                        return
                except ValueError:
                    print("Non-numeric value entered for wins, losses, kills, number of friends, or money spent.")
                    return
    
                # checks if username already exists
                # variable key is the one getting queried
                username_response = table.query(
                    KeyConditionExpression=Key('username').eq(username.value)
                )
                if username_response['Items']:
                    print("Username already in use. Please enter a new one.")
                    return
    
                # Scan the table to check if email already exists
                email_exists = False
                response = table.scan(
                    FilterExpression=Attr('email').eq(email.value)
                )
                for item in response['Items']:
                    if item['email'] == email.value:
                        email_exists = True
                        break
    
                if email_exists:
                    print("Email already in use. Please enter a new one.")
                    return
    
                # Insert the new profile into DynamoDB
                profile_data = {
                    "username": username.value,
                    "email": email.value,
                    "wins": wins.value,
                    "losses": losses.value,
                    "games_played": wins.value + losses.value,
                    "kills": kills.value,
                    "number_of_friends": number_of_friends.value,
                    "favorite_weapon": favorite_weapon.value,
                    "money_spent": Decimal(str(money_spent.value)),
                    "favorite_character": favorite_character.value,
                }
                table.put_item(Item=profile_data)
                print("Profile Inserted")
    
        insert_button = widgets.Button(description="Insert")
        insert_button.on_click(insert)
    
        insert_items = [
            labeled_input('Username: ', username),
            labeled_input('Email: ', email),
            labeled_input('Wins: ', wins),
            labeled_input('Losses: ', losses),
            labeled_input('Kills: ', kills),
            labeled_input('Number of Friends: ', number_of_friends),
            labeled_input('Favorite Weapon: ', favorite_weapon),
            labeled_input('Money Spent: ', money_spent),
            labeled_input('Favorite Character: ', favorite_character),
            insert_button
        ]
    
        display(form_output)
        for item in insert_items:
            display(item)
        display(message_output)
    

    def valid_delete():
        output = widgets.Output()
        output.clear_output(wait=True)
    
        username = widgets.Text()
        labeled_username = labeled_input("Username: ", username)
    
        delete_button = widgets.Button(description="Delete")
    
        def delete(submission):
            with output:
                output.clear_output()
                # Check if username exists
                response = table.query(
                    KeyConditionExpression=Key('username').eq(username.value))
        
                if not response['Items']:
                    print("No profile found with the given username.")
                    return
        
                # Delete the profile(s) from DynamoDB
                for item in response['Items']:
                    # Use both username and email as key elements to delete the item
                    table.delete_item(
                        Key={'username': item['username'], 'email': item['email']}
                    )
                print("Profile(s) Deleted")

        delete_button.on_click(delete)
    
        show_widgets = [
            labeled_username,
            delete_button,
            output
        ]
        display(*show_widgets)


    # valid_modify is function for if the user selects modify
    def valid_modify():
    
        # widgets that will be used if user selects modify
        output = widgets.Output()
        numeric_conditions_container = widgets.VBox([])
        favorite_conditions_container = widgets.VBox([])
        modify_button = widgets.Button(description="Modify")
    
        username = widgets.Text()
        email = widgets.Text()
        labeled_username = labeled_input("Username: ", username)
        labeled_email = labeled_input("Email: ", email)
    
        options = ['username', 'email', 'wins', 'losses', 'kills', 'number_of_friends',
                   'money_spent', 'favorite_character', 'favorite_weapon']
        fields_to_modify = widgets.SelectMultiple(
            options=options,
            disabled=False
        )
        labeled_fields_to_modify = labeled_input('Column(s) to Modify: ', fields_to_modify)
    
        modify_fields_container = widgets.VBox([])
    
        # function so that user can select which field(s) they want to change and a textbox will
        # appear where they can input the new value for the corresponding field(s) they want to change
        def fields_modify(change):
            children = []
            for field in change['new']:
                value_input = widgets.Text()
                labeled_value_input = labeled_input(f"New {field}: ", value_input)
                children.append(labeled_value_input)
            modify_fields_container.children = children
    
        fields_to_modify.observe(fields_modify, names='value')

    
        # modify function will modify the profile(s) or column(s) in database; modification is based on 
        # the username entered by the user. user will also be given option
        # to select which fields they want to modify and input the new values.
            
        def modify(submission):
            with output:
                clear_output()
                query_conditions = []

                # key = {'username': username.value, 'email': email.value}
                key = {'username': username.value, 'email': email.value}
                
                # Check if the key is valid
                if not key: 
                    print("Key is missing.")
                    return
                if not username.value:
                    print("Please provide a valid username.")
                    return
        
                # Add username condition if provided
                if username.value:
                    query_conditions.append({"username": username.value})
                            
                
                # Construct FilterExpression based on query conditions
                query = {"FilterExpression": " AND ".join([f"({condition})" for condition in query_conditions])} if query_conditions else {}
        
                if not query_conditions:
                    print("Specify username or condition to delete.")
                    return
                
                modify_values = {}
                
                # Iterate through fields to modify and their new values
                for i in modify_fields_container.children:
                    label_widget = i.children[0]
                    text_widget = i.children[1]
                            
                    field = label_widget.value.replace("New", "").replace(":", "").strip().lower()
                    original_value = text_widget.value.strip()
                
                    if field in ['username', 'email'] and not original_value:
                        print(f"Cannot leave '{field}' blank")
                        return
                                
                    if field in ['wins', 'losses',  'kills', 'number_of_friends']:
                        try:
                            value = int(original_value)
                            if value < 0:
                                print(f"New Value for {field} cannot be negative.")
                                return
                
                        except ValueError:
                            print(f"New Value for {field} must be a integer greater than or equal to 0.")
                            return
                    elif field in ['money_spent']:
                        try:
                            value = int(original_value)
                            if value < 0:
                                print("New Value for 'money_spent' cannot be negative.")
                                return
                
                        except ValueError:
                            print(print("New Value for 'money_spent' must be a number greater than or equal to 0."))
                            return
                    else:
                        value = original_value
                                    
                    modify_values[field.strip()] = value

                
                # Calculate games played based on wins and losses if they are modified
                if 'wins' in modify_values or 'losses' in modify_values:
                    response = table.query(KeyConditionExpression=Key('username').eq(username.value))
                    items = response['Items']
                    if items:
                        current_profile = items[0]  # Assuming there's only one item matching the key
                    else:
                        current_profile = {}
                    # current_profile = response.get('Item', {})
                                
                    current_wins = int(modify_values.get('wins', str(current_profile.get('wins', {'N': '0'}))))
                    current_losses = int(modify_values.get('losses', str(current_profile.get('losses', {'N': '0'}))))

                    games_played = current_wins + current_losses
                    modify_values['games_played'] = {'N': str(games_played)}
        
                if modify_values:
                    update_expression = "SET " + ", ".join([f"#{k.strip()} = :{k.strip()}" for k in modify_values.keys()])
                    expression_attribute_values = {f":{k.strip()}": v for k, v in modify_values.items()}
                    expression_attribute_names = {"#{}".format(k.strip()): k.strip() for k in modify_values.keys()}
                
                    # print("Update Expression:", update_expression)
                    # print("Expression Attribute Values:", expression_attribute_values)
                    # print("Expression Attribute Names:", expression_attribute_names)
                
                    try:
                        # Attempt to update the item in the database
                        response = table.update_item(
                            Key=key,
                            UpdateExpression=update_expression,
                            ExpressionAttributeValues=expression_attribute_values,
                            ExpressionAttributeNames=expression_attribute_names,
                            ReturnValues="UPDATED_NEW"
                        )
                        if 'Attributes' in response:
                            print("Modified profile(s)".format(response['Attributes']))
                        else:
                            print("No profiles were modified")
                    except Exception as e:
                        print("Failed to update item:", e)


    # Attach the callback function to the "Modify" button
    # modify_button.on_click(modify_button_clicked)
    
        modify_button.on_click(modify)
    
        # widgets that will be displayed when modify is selected
        show_widgets = [
            labeled_fields_to_modify,
            labeled_username,
            labeled_email,
            modify_fields_container,
            modify_button,
            output
        ]
        display(*show_widgets)
    
    
    # used to translate the symbols to the DynamoDB syntax
    def numeric_query(field, condition, value):
        operators = {
            ">": "gt",
            "<": "lt",
            ">=": "gte",
            "<=": "lte",
            "=": "eq",
            "≠": "ne"
        }
        return Attr(field).contains({operators[condition]: Decimal(str(value))})

    def clear(click):
        with output:
            clear_output()
    
    submit.on_click(choice)
    clear_button.on_click(clear)

    buttons = widgets.HBox([submit, clear_button])
    display(labeled_action, buttons, output)

In [16]:
def search_mongo():

    # to have labels in HTMl so it fits on the screen, make it bold too
    def labeled_input(description, widget):
        return widgets.VBox([widgets.Label(value=f"{description}"), widget])

    # widgets that will be used for this function
    options = [ 'all', 'username', 'email', 'games_played', 'wins', 'losses',
                  'kills', 'number_of_friends', 'favorite_weapon', 'money_spent', 
                  'favorite_character']
    attributes = widgets.SelectMultiple(
        options = options,
        value = ['all'],
        disabled = False
    )
    labeled_attributes = labeled_input("Attributes(hold command to select multiple): ", attributes)

    operator = widgets.Dropdown(
        options = ['Select', 'AND', 'OR'],
        value = 'Select',
        disabled = False
    )
    labeled_operator = labeled_input("Operator: ", operator)


    number_of_numerical_conditions = widgets.Dropdown(
        options = [0,1,2,3,4,5,6],
        value = 0,
        disabled = False,
    )
    labeled_number_of_numerical_conditions = labeled_input("Number of Numeric Conditions: ", number_of_numerical_conditions)


    number_of_favorite_conditions = widgets.Dropdown(
        options = [0,1,2],
        value = 0,
        disabled = False,
    )
    labeled_number_of_favorite_conditions = labeled_input("Number of Favorite Conditions: ", number_of_favorite_conditions)

    
    search_container_numerical = widgets.VBox([])
    search_container_favorite= widgets.VBox([])
    
    search_button = widgets.Button(
        description = 'Search',
        button_style = '',
        tooltip = "Click to Search",
        icon = 'search'
    )
    clear_button = widgets.Button(
        description = 'Clear', 
        button_style = '', 
        tooltip = 'Click to clear search', 
        icon = 'remove')
    output = widgets.Output()

    # function so user can't select all along with the other options, only all, or only the other options
    def select_all(change):
        if 'all' in change['new']:
            if len(change['new']) > 1:
                if 'all' in change['old']:
                    attributes.value = tuple(set(change['new']) - {'all'})
                else:
                    attributes.value = tuple(options[1:])
            else:
                attributes.value = ('all',)
    
    attributes.observe(select_all, 'value')
    
    # this function allows user to select how many numeric conditions they would like; numeric fields include
    # 'wins', 'losses', 'games played', 'kills', 'number of friends', and 'money spent'; function will 
    # display number of numeric condition widgets according to the number selected by user; unlike previous
    # num_of_numerical_conditions, user will also have the option to search for max or min values; 
    # when that occurs, the value widget will disappear
    def num_of_numerical_conditions(change):
        conditions = change['new'] 
        children = []
        for _ in range(conditions):
            field = widgets.Dropdown(
                options = ['select', 'wins', 'losses', 'games_played', 'kills', 'number_of_friends', 'money_spent'],
                disabled = False
            )
            condition = widgets.Dropdown(
                options = ['select', '=', '>', '>=', '<', '<=', 'max', 'min'],
                disabled = False
            )
            value = widgets.FloatText(
                value = 0,
                disabled = False
            )
            value_container = widgets.VBox(children = [widgets.HTML(value = "<b>Value:</b>"), value])

            def value_visibile(change, container = value_container):
                if change['new'] in ['max', 'min']:
                    container.layout.display = 'none'
                else:
                    container.layout.display = ''

            condition.observe(value_visibile, names = 'value')

            
            children.append(widgets.HBox([labeled_input('Field: ', field), labeled_input('Condition: ', condition), value_container]))
        search_container_numerical.children = children
    
    num_of_numerical_conditions({'new': number_of_numerical_conditions.value})
    number_of_numerical_conditions.observe(num_of_numerical_conditions, names = 'value')

    # this function allows user to select how many favorite conditions they would like; favorite fields include
    # 'favorite character' and 'favorite weapon'; function will 
    # display number of favorite condition widgets according to the number selected by user 
    # unlike previous num_of_favorite_conditions, user will also have the option to search for most 
    # or least common values; when that occurs, the value widget will disappear
    def num_of_favorite_conditions(change):
        conditions = change['new'] 
        children = []
        for _ in range(conditions):
            field = widgets.Dropdown(
                options = ['select', 'favorite_character', 'favorite_weapon'],
                disabled = False
            )
            condition = widgets.Dropdown(
                options = ['select', '=','≠'],
                disabled = False
            )
            value = widgets.Text(
                disabled = False
            )
            value_container = widgets.VBox(children = [widgets.HTML(value = "<b>Value:</b>"), value])
            
            children.append(widgets.HBox([labeled_input('Field: ', field), labeled_input('Condition: ', condition), value_container]))
        search_container_favorite.children = children

    num_of_favorite_conditions({'new': number_of_favorite_conditions.value})
    number_of_favorite_conditions.observe(num_of_favorite_conditions, names = 'value')

    # this function will will search for profiles based on the numeric and/or the favorite conditions; numeric
    # conditions give user the option to select the numeric fields described above, as well as one of the conditions,
    # <, >, <=, >=, =, max or min and the value(except for max and min) to compare and return profiles based on the conditions; 
    # favorite conditions give the user the option to select the favorite fields as described above, as well as one of the conditions,
    # =, ≠, most common or least common and the value to compare to(except for most/least common) to compare and return profiles based on the conditions; 
    # user can select multiple conditions from either favorite or numeric and select AND or OR to join them; user will also be given option
    # to select which fields they want to see or select all to see all the fields
    # finally, function will tell user how many profile(s)/field(s) were founding matching the conditions
    def search_button_clicked(search):
        with output:
            clear_output()
            selected_attributes = list(attributes.value)
            if 'all' in selected_attributes:
                selected_attributes = options[1:]
            fields = {field: 1 for field in selected_attributes}
            fields['_id'] = 0
            
            query_conditions = []
            logical_operator = "$and"
            if operator.value in ['AND', 'OR']:
                logical_operator = "$and" if operator.value == 'AND' else "$or"

            
            max_min = []
            if number_of_numerical_conditions.value > 0:
                for search_condition in search_container_numerical.children:
                    field_widget, condition_widget, value_widget = search_condition.children
                    field = field_widget.children[1].value
                    condition = condition_widget.children[1].value
                    value =  value_widget.children[1].value
                    if field == 'select' or condition == 'select':
                        print("No field or condition selected, please select one.")
                        return
                    elif field != 'select' and condition in ['max', 'min']:
                        max_min.append((field,condition))
                    else:
                        subquery = numeric_query(field, condition, value)
                        query_conditions.append(subquery)

            favorite_max_min = []
            if number_of_favorite_conditions.value > 0:
                for search_condition in search_container_favorite.children:
                    field_widget,condition_widget,value_widget = search_condition.children
                    field = field_widget.children[1].value
                    condition = condition_widget.children[1].value
                    value = value_widget.children[1].value
                    
                    if field == 'select' or condition == 'select':
                        print("No field or condition selected, please select one.")
                        return
  
                    elif field != 'select' and condition != 'select':
                        subquery = numeric_query(field, condition, value)
                        query_conditions.append(subquery)
 
                    else:
                        subquery = numeric_query(field, condition, value)
                        query_conditions.append(subquery)


            for field, condition in max_min:
                sort_order = -1 if condition == 'max' else 1
                pipe = [
                    {"$match": {field: {"$exists": True, "$ne": None}}},
                    {"$sort": {field: sort_order}},
                    {"$limit": 1}
                ]
                result = list(collection.aggregate(pipe))
                if result and field in result[0]:
                    max_or_min = result[0][field]
                    match = {"$eq": max_or_min}
                    query_conditions.append({field: max_or_min})
                else:
                    print("No value found")
                
                        
            if query_conditions:
                query = {logical_operator: query_conditions} if len(query_conditions) > 1 else(query_conditions[0] if query_conditions
                                                                                              else {})
                profiles = collection.find(query, fields)
                count = collection.count_documents(query)
                if count == 0:
                    print("No profiles found matching you search")
                else:
                    print(f"{count} profile(s) found matching your search")
                    for profile in profiles:
                        out = ', '.join([f"{key}: {value}" for key, value in profile.items() if key != '_id'])
                        print(out)
    search_button.on_click(search_button_clicked)

    # display the widgets for this function
    buttons = widgets.HBox([search_button, clear_button])
    display(labeled_operator, labeled_number_of_numerical_conditions, search_container_numerical, 
            labeled_number_of_favorite_conditions,  search_container_favorite, labeled_attributes,
            buttons, output)

    # clear to reset the values and clear the output
    def clear(click):
        with output:
            clear_output()
        number_of_numerical_conditions.value = 0
        num_of_numerical_conditions({'new': 0})
        number_of_favorite_conditions.value = 0
        num_of_favorite_conditions({'new': 0})


    clear_button.on_click(clear)

     # used to translate the symbols to the Mongo syntax like above
    def numeric_query(field, condition, value):
        operators = {
            ">":"$gt",
            "<":"$lt",
            ">=":"$gte",
            "<=":"$lte",
            "=":"$eq",
            "≠":"$ne"
        }
        return {field: {operators[condition]: value}}


In [17]:
def search_dynamo():

    # Function to create labeled input widgets
    def labeled_input(description, widget):
        return widgets.VBox([widgets.Label(value=f"{description}"), widget])

    # Widgets setup
    options = ['all', 'username', 'email', 'games_played', 'wins', 'losses',
               'kills', 'number_of_friends', 'favorite_weapon', 'money_spent',
               'favorite_character']
    attributes = widgets.SelectMultiple(
        options=options,
        value=['all'],
        disabled=False
    )
    labeled_attributes = labeled_input("Attributes (hold command to select multiple): ", attributes)

    operator = widgets.Dropdown(
        options=['Select', 'AND', 'OR'],
        value='Select',
        disabled=False
    )
    labeled_operator = labeled_input("Operator: ", operator)

    number_of_numerical_conditions = widgets.Dropdown(
        options=[0, 1, 2, 3, 4, 5, 6],
        value=0,
        disabled=False,
    )
    labeled_number_of_numerical_conditions = labeled_input("Number of Numeric Conditions: ",
                                                           number_of_numerical_conditions)

    number_of_favorite_conditions = widgets.Dropdown(
        options=[0, 1, 2],
        value=0,
        disabled=False,
    )
    labeled_number_of_favorite_conditions = labeled_input("Number of Favorite Conditions: ",
                                                           number_of_favorite_conditions)

    search_container_numerical = widgets.VBox([])
    search_container_favorite = widgets.VBox([])

    search_button = widgets.Button(
        description='Search',
        button_style='',
        tooltip="Click to Search",
        icon='search'
    )
    clear_button = widgets.Button(
        description='Clear',
        button_style='',
        tooltip='Click to clear search',
        icon='remove')
    output = widgets.Output()

    def select_all(change):
        if 'all' in change['new']:
            if len(change['new']) > 1:
                if 'all' in change['old']:
                    attributes.value = tuple(set(change['new']) - {'all'})
                else:
                    attributes.value = tuple(options[1:])
            else:
                attributes.value = ('all',)

    attributes.observe(select_all, 'value')

    def num_of_numerical_conditions(change):
        conditions = change['new']
        children = []
        for _ in range(conditions):
            field = widgets.Dropdown(
                options=['select', 'wins', 'losses', 'games_played', 'kills', 'number_of_friends', 'money_spent'],
                disabled=False
            )
            condition = widgets.Dropdown(
                options=['select', '=', '>', '>=', '<', '<=', '≠'],
                disabled=False
            )
            value = widgets.FloatText(
                value=0,
                disabled=False
            )
            value_container = widgets.VBox(children=[widgets.HTML(value="<b>Value:</b>"), value])

            def value_visible(change, container=value_container):
                if change['new'] in ['max', 'min']:
                    container.layout.display = 'none'
                else:
                    container.layout.display = ''

            condition.observe(value_visible, names='value')

            children.append(widgets.HBox([labeled_input('Field: ', field), labeled_input('Condition: ', condition),
                                          value_container]))
        search_container_numerical.children = children

    num_of_numerical_conditions({'new': number_of_numerical_conditions.value})
    number_of_numerical_conditions.observe(num_of_numerical_conditions, names='value')

    def num_of_favorite_conditions(change):
        conditions = change['new']
        children = []
        for _ in range(conditions):
            field = widgets.Dropdown(
                options=['select', 'favorite_character', 'favorite_weapon'],
                disabled=False
            )
            condition = widgets.Dropdown(
                options=['select', '=', '≠'],
                disabled=False
            )
            value = widgets.Text(
                disabled=False
            )
            value_container = widgets.VBox(children=[widgets.HTML(value="<b>Value:</b>"), value])

            children.append(widgets.HBox([labeled_input('Field: ', field), labeled_input('Condition: ', condition),
                                          value_container]))
        search_container_favorite.children = children

    num_of_favorite_conditions({'new': number_of_favorite_conditions.value})
    number_of_favorite_conditions.observe(num_of_favorite_conditions, names='value')

    def search_button_clicked(search):
        with output:
            output.clear_output()
            selected_attributes = list(attributes.value)
            if 'all' in selected_attributes:
                selected_attributes = options[1:]
            projection_expression = ', '.join(selected_attributes)

            logical_operator = "AND"
            if operator.value in ['AND', 'OR']:
                logical_operator = "AND" if operator.value == 'AND' else "OR"

            query_conditions = []

            if number_of_numerical_conditions.value > 0:
                for search_condition in search_container_numerical.children:
                    field_widget, condition_widget, value_widget = search_condition.children
                    field = field_widget.children[1].value
                    condition = condition_widget.children[1].value
                    value = value_widget.children[1].value

                    if field != 'select' and condition != 'select':
                        query_conditions.append(numeric_query(field, condition, value))

            if number_of_favorite_conditions.value > 0:
                for search_condition in search_container_favorite.children:
                    field_widget, condition_widget, value_widget = search_condition.children
                    field = field_widget.children[1].value
                    condition = condition_widget.children[1].value
                    value = value_widget.children[1].value

                    if field != 'select' and condition != 'select':
                        query_conditions.append(favorite_query(field, condition, value))

            if query_conditions:
                if logical_operator == "AND":
                    filters = reduce(lambda x, y: x & y, query_conditions)
                else:
                    filters = reduce(lambda x, y: x | y, query_conditions)

                response = table.scan(
                    ProjectionExpression=projection_expression,
                    FilterExpression=filters
                )

                items = response['Items']
                print(f"{len(items)} item(s) found matching your search.")
                for item in items:
                    print(item)

    search_button.on_click(search_button_clicked)

    buttons = widgets.HBox([search_button, clear_button])
    display(labeled_operator, labeled_number_of_numerical_conditions, search_container_numerical,
            labeled_number_of_favorite_conditions, search_container_favorite, labeled_attributes,
            buttons, output)

    def clear(click):
        with output:
            output.clear_output()
        number_of_numerical_conditions.value = 0
        num_of_numerical_conditions({'new': 0})
        number_of_favorite_conditions.value = 0
        num_of_favorite_conditions({'new': 0})

    clear_button.on_click(clear)


def numeric_query(field, condition, value):
    operators = {
        ">": "gt",
        "<": "lt",
        ">=": "gte",
        "<=": "lte",
        "=": "eq",
        "≠": "ne"
    }

    value = Decimal(str(value))
    
    return getattr(Key(field), operators[condition])(value)


def favorite_query(field, condition, value):
    if condition == "=":
        return Attr(field).eq(value)
    elif condition == "≠":
        return Attr(field).ne(value)

In [18]:
def selection():
    # to have labels in HTMl so it fits on the screen
    def labeled_input(description, widget):
        return widgets.VBox([widgets.Label(value=f"{description}"), widget])

    # widgets that will be used, including dropdown to select database and select operation
    database_selection = widgets.Dropdown(
        options = ['MongoDB', 'DynamoDB'],
        value = 'MongoDB',
        disabled = False
    )
    database_selection_labeled = labeled_input("Select Database", database_selection)

    operation_selection= widgets.Dropdown(
        options = ['View', 'Change', 'Search'],
        value = 'View',
        disabled = False
    )
    operation_selection_labeled = labeled_input("Select Operation", operation_selection)
    
    operation_button = widgets.Button(
        description = 'Search', 
        button_style = '', 
        tooltip = 'Click to clear search', 
    )
    output = widgets.Output()

    # function that executes the proper function based on the database the user selects and the operation the user selects
    def operation_execution(operation):
        with output:
            output.clear_output()
            if database_selection.value == 'MongoDB':
                if operation_selection.value == 'View':
                    view_mongo()
                elif operation_selection.value == 'Change':
                    change_mongo()
                elif operation_selection.value == 'Search':
                    search_mongo()
            elif database_selection.value == 'DynamoDB':
                if operation_selection.value == 'View':
                    view_dynamo()
                elif operation_selection.value == 'Change':
                    change_dynamo()
                elif operation_selection.value == 'Search':
                    search_dynamo()

    # display buttons and ensure when the operation button is clicked, the function executse
    operation_button.on_click(operation_execution)
    display(database_selection_labeled, operation_selection_labeled, operation_button, output)

selection()

Button(description='Search', style=ButtonStyle(), tooltip='Click to clear search')

Output()